<a href="https://colab.research.google.com/github/louzeiro/Roteamento-Veiculo/blob/main/Ortools_VRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook desenvolvido durante os estudos da ferramenta Or-Tools<br>
Seção: Roteamento<br>
Subseção: Vehicle Rounting Problem


Fonte: https://developers.google.com/optimization/routing/vrp


# Definição do problema

Neste notebook foi implementada para a solução do problema de roteamento de veículo com o Or-Tools. No exemplo de uso, foi definido o conjunto de pontos do problema, conforme a imagem abaixo. Neste exemplo o objetivo é otimizar a visita dos nós usando 4 veículos. Contudo, é necessário adicionar uma restrição de distância máxima a ser percorrida por cada veículo ao problema.
<p>
  <a href="exemplo">
    <img src="https://developers.google.com/optimization/images/routing/vrp.svg">
  </a>
</p>


# Conjunto de dados

In [2]:
"""
 Dados do problema: conjunto das distâncias entre os pontos, incluindo o
 deposito. Nesse caso, não há diferênça no percurso de ida e volta.
 O ponto de partida e de chegada será o depósito.
 O problema utiliza 4 veículos para realizar a visita nos pontos
"""
def create_data_model():
    data = {}
    data['distance_matrix'] = [
        [ # Ponto 0 - deposito
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

# Definindo o modelo de roteamento

In [8]:
import os
os.system('pip install ortools') #instalando a biblioteca no ambiente de desenvolvimento

from ortools.constraint_solver import pywrapcp 
data = create_data_model()                                           # carregando os dados do problema
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), # número de pontos
                                       data['num_vehicles'],         # total de veículos/caixeiros  
                                       data['depot'])                # índece da origem/depósito
routing = pywrapcp.RoutingModel(manager)                             # criando o modelo


# Defindo a função que retornará a distância entre dois pontos

Essa função é necessária, pois o Ortools trabalha com um sistema diferente de referência.

In [9]:
"""Retorna a distância entre os pontos"""
def distance_callback(from_index, to_index):
  from_node = manager.IndexToNode(from_index)   # convertendo o index interno do ortools
  to_node = manager.IndexToNode(to_index)       # para o index de acesso à matrix
  return data['distance_matrix'][from_node][to_node]

"""Realiza a chamada e a registra no solve """
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [5]:
data['distance_matrix'][1][2]

684

# Definindo o custo

Neste exemplo será utilizado a distância entre os pontos como o custo do problema. Assim, será utilizada a variável *transit_callback_index* que é a referência interna do solve para a função *distance_callback*, que retorna a distância.

In [10]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Definindo os parâmetros e a Heurística da solução


In [13]:
from ortools.constraint_solver import routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Adicionando a dimensão distância ao modelo

In [12]:
dimension_name = 'Distance'
max_distance_m = 3000

routing.AddDimension(
    transit_callback_index,
    0,      # indica se haverá um período de parada
    max_distance_m, # Máximo a ser percorrido      
    True,           # para iniciar a contagem com 0
    dimension_name 
)

distance_dimension = routing.GetDimensionOrDie(dimension_name)
"""
O método SetGlobalSpanCostCoeficiente define um coeficiente grande (100) para 
a extensão global das rotas, que neste exemplo é o máximo das distâncias das 
rotas. Isso torna a extensão global o fator predominante na função objetivo, 
de modo que o programa minimiza o comprimento da rota mais longa.
"""
distance_dimension.SetGlobalSpanCostCoefficient(100)


# Exibindo os resultados obtidos

In [48]:
def print_solution(data, manager, routing, solution):
    print('Objetivo: {} metros'.format(solution.ObjectiveValue()))
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']): # para cada veículo
      index = routing.Start(vehicle_id)
      plan_output = 'Rota para o veículo {}:\n'.format(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          plan_output += ' {} ->'.format(manager.IndexToNode(index))
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(
              previous_index,
               index, 
               vehicle_id)
          
      plan_output += ' {}\n'.format(manager.IndexToNode(index))
      plan_output += 'Rota para a distância percorrida: {} metros\n'.format(route_distance)
      print(plan_output)
      max_route_distance = route_distance + max_route_distance
    print('A distância total percorrida pelos {} veículos foi: {} metros'.format(data['num_vehicles'], max_route_distance))

    

# Main

In [49]:
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print_solution(data, manager, routing, solution)
else:
  print('Este problema não possue solução para as configurações apresentadas')

Objetivo: 161408 metros
Rota para o veículo 0:
 0 -> 9 -> 10 -> 16 -> 14 -> 0
Rota para a distância percorrida: 1552 metros

Rota para o veículo 1:
 0 -> 13 -> 15 -> 11 -> 12 -> 0
Rota para a distância percorrida: 1552 metros

Rota para o veículo 2:
 0 -> 7 -> 1 -> 4 -> 3 -> 0
Rota para a distância percorrida: 1552 metros

Rota para o veículo 3:
 0 -> 8 -> 6 -> 2 -> 5 -> 0
Rota para a distância percorrida: 1552 metros

A distância total percorrida pelos 4 veículos foi: 6208 metros


# Salvando as rotas em um array

In [47]:
def get_routes(solution, routing, manager):
  routes = [] 
  for route_nbr in range(routing.vehicles()): # para cada veículo do conjunto de rotas
    index = routing.Start(route_nbr)      # Ponto de partida (depósito)
    route = [manager.IndexToNode(index)]  # Convertende e adicionando o index no array
    while not routing.IsEnd(index):       # Enquanto não for o último índice
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index)) 
    routes.append(route)
  return routes

#exibindo a rota final
routes = get_routes(solution, routing, manager)
for i, route in enumerate(routes):
  print('Rota obtida com a solução para o veículo', i,'=', route)

Rota obtida com a solução para o veículo 0 = [0, 9, 10, 16, 14, 0]
Rota obtida com a solução para o veículo 1 = [0, 13, 15, 11, 12, 0]
Rota obtida com a solução para o veículo 2 = [0, 7, 1, 4, 3, 0]
Rota obtida com a solução para o veículo 3 = [0, 8, 6, 2, 5, 0]
